# 第七章 练习

## 8. MNIST数据集上的投票集成

In [1]:
#setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data1 = pd.read_csv('../datasets/MNIST/mnist_784.csv')
#data2 = pd.read_csv('../datasets/MNIST/mnist_train.csv')
#data3 = pd.read_csv('../datasets/MNIST/mnist_test.csv')

In [3]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 785 entries, pixel1 to class
dtypes: int64(785)
memory usage: 419.2 MB


In [4]:
data1.describe()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
count,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,70000.0,...,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.0,70000.0,70000.0,70000.0,70000.000000
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.099543,0.046629,0.016614,0.012957,0.001714,0.0,0.0,0.0,0.0,4.452429
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.256304,2.783732,1.561822,1.553796,0.320889,0.0,0.0,0.0,0.0,2.890195
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,2.000000
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,4.000000
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,7.000000
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,254.000000,253.000000,253.000000,254.000000,62.000000,0.0,0.0,0.0,0.0,9.000000


In [5]:
data1.head(8)

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [6]:
def split_dataset(data, val_size, test_size):
    from sklearn.model_selection import train_test_split
    X_data_val, X_data_test, y_data_val, y_data_test = train_test_split(data.iloc[:,:-1], data.iloc[:,-1], test_size=test_size)
    X_data_train, X_data_val, y_data_train, y_data_val = train_test_split(X_data_val, y_data_val, test_size=val_size)
    return X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test

In [7]:
X_train, y_train, X_val, y_val, X_test, y_test = split_dataset(data1, 50000, 10000)

In [8]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

rf_clf = RandomForestClassifier(n_estimators=50, 
                                max_depth=None, 
                                min_samples_leaf=2, 
                                bootstrap=True, 
                                max_samples=0.5,
                                n_jobs=-1
                               )
et_clf = ExtraTreesClassifier(n_estimators=50,
                              max_depth=None,
                              min_samples_leaf=2,
                              bootstrap=True,
                              max_samples=0.5,                              
                              n_jobs=-1
                             )
svc = SVC(kernel='rbf')
vt_clf = VotingClassifier(estimators=[('rf', rf_clf), ('et', et_clf), ('svc', svc)],
                         voting='hard',
                         n_jobs=-1)

In [9]:
clfs = (svc, rf_clf, et_clf)

In [10]:
scores = []
for clf in clfs:
    clf.fit(X_train, y_train)
    y_val_pred = clf.predict(X_val)
    y_pred = clf.predict(X_test)
    val_score = accuracy_score(y_val_pred, y_val)
    test_score = accuracy_score(y_pred, y_test)
    scores.append([clf.score(X_train, y_train), val_score, test_score])

In [11]:
print(scores)

[[0.9839, 0.961, 0.9606], [0.9918, 0.93546, 0.9347], [0.9884, 0.93206, 0.9321]]


In [12]:
clf1 = vt_clf
clf1.fit(X_train, y_train)
y_val_pred = clf1.predict(X_val)
y_pred = clf.predict(X_test)
val_score = accuracy_score(y_val_pred, y_val)
test_score = accuracy_score(y_pred, y_test)
scores.append([clf1.score(X_train, y_train), val_score, test_score])

In [13]:
print(scores)

[[0.9839, 0.961, 0.9606], [0.9918, 0.93546, 0.9347], [0.9884, 0.93206, 0.9321], [0.9916, 0.94854, 0.9321]]


In [14]:
vt_clf

VotingClassifier(estimators=[('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=0.5,
                                                     min_impurity_decrease=0.0,
                                                     min_impurity_split=None,
                                                     min_samples_leaf=2,
                                                     min_samples_split=2,
                                                     min_weight_fraction_leaf=0.0,
         

In [15]:
vt2_clf = vt_clf

In [16]:
del vt2_clf.estimators_[2]

In [20]:
vt2_clf.estimators_

[RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=0.5,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=2, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                        oob_score=False, random_state=None, verbose=0,
                        warm_start=False),
 ExtraTreesClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                      criterion='gini', max_depth=None, max_features='auto',
                      max_leaf_nodes=None, max_samples=0.5,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=-1,
                      

In [18]:
vt2_clf.voting='soft'

In [19]:
vt2_clf.score(X_train, y_train), vt_clf.score(X_val, y_val), vt_clf.score(X_test, y_test)

(0.9929, 0.94144, 0.9385)